In [42]:
from sklearn.cluster import KMeans

In [43]:
file_name = './test2.data'
article = 'https://news.mail.ru/economics/39536541/'

In [30]:
# Первый вариант решения задачи:

# Для каждого пользователя находим все статьи, которые он посещал. 
# Далее находим расстояние от пользователей, посещавших нужную статью, до остальных. 
# За расстояние берём количество отличающихся статей пользователей. 
# Наиболее близких пользователей берём как пользователей с наименьшей суммой расстояний до "нужных" пользователей. 

In [36]:
des_users = set()  # Нужные пользователи
dist_urls = set()  # Уникальные урлы
user_url = {}  # {пользователь: посещённые статьи}
with open(file_name) as f:
    for line in f:
        user, url = line.split()[0], line.split()[1]
        if user not in user_url:
            user_url[user] = set()
        user_url[user].add(url)
        
        dist_urls.add(url)
        if url == article:
            des_users.add(user)


In [24]:
distance = {}  # {пользователь: сумма расстояний до нужных пользователей}
for i, des_user in enumerate(des_users):
    for j, user in enumerate(user_url):
        if user in des_users:  # В топе не будем учитывать пользователей, посещавших нужную статью
            continue

        if i == 0:
            distance[user] = len(user_url[user].symmetric_difference(user_url[des_user]))
        else:
            distance[user] += len(user_url[user].symmetric_difference(user_url[des_user]))

In [25]:
sort = sorted(distance.items(), key=lambda item: item[1])[:10000]

In [27]:
# Получившийся топ пользователей
top = [x[0] for x in sort]

In [31]:
# Этот способ решения плохо работает на тестовых данных, так как большинство пользователей посещали только одну статью

# Второй вариант решения:

# Разделим все урлы на кластеры с помощью kmeans. 
# Каждому урлу добавим список фичей: 
#     принадлежность к категории, количество сегментов урла, содержит 'comment', содержит 'gallery'
# И дальше расчитаем расстояние, как количество пересекающихся классов у пользователей 
# (в топ войдут пользователи с наибольшим расстоянием)


In [39]:
# Находим все категории статей
topic_names = []
for i in dist_urls:
    a = i.split('/')
    if a[3] not in topic_names:
        topic_names.append(a[3])

print(topic_names)

['politics', 'amp', 'economics', 'incident', 'tag', 'society', 'foto', 'currency', 'infographics', 'landing', 'video', 'company', 'wall-127229518_31843', 'wall-182116422_356', 'media', 'ufa1ru', 'video-182116422_456239026']


In [40]:
# Удаляем специфичные категории, считаем, что это категория other 
topic_names.remove('video-182116422_456239026')
topic_names.remove('wall-182116422_356')
topic_names.remove('wall-127229518_31843')

In [41]:
user_url = set()
des_users = set()
url_features = {} # {урл: список фичей}

with open(file_name) as f:
    for line in f:
        user, url = line.split()[0], line.split()[1]
        user_url.add((user, url))
        if url == article:
            des_users.add(user)
        
        if url not in url_features:
            split_url = url.split('/')
            k_sections = len(split_url) - 2
            topic = [0] * (len(topic_names) + 1)
            if '-' in split_url[3]:
                topic[-1] = 1
            else:
                topic[topic_names.index(split_url[3])] = 1
                
            comments = 0
            if 'comments' in url:
                comments = 1
            gallery = 0
            if 'gallery' in url:
                gallery = 1
                
            url_features[url] = [k_sections, comments, gallery] + topic

In [49]:
features = list(url_features.values())
kmeans = KMeans(n_clusters=18).fit(features)
url_class = list(kmeans.labels_)

In [50]:
# Определяем для каждого пользователя список классов статей, которые они посещали
user_class = {}
for i in user_url:
    user, url = i[0], i[1]
    
    if user not in user_class:
        user_class[user] = set()
        
    val = url_features[url]
    user_class[user].add(url_class[features.index(val)])

In [51]:
distance = {}
for i, des_user in enumerate(des_users):
    for j, user in enumerate(user_class.keys()):
        if user in des_users:
            continue

        if i == 0:
            distance[user] = len(user_class[user].intersection(user_class[des_user]))
        else:
            distance[user] += len(user_class[user].intersection(user_class[des_user]))

In [52]:
sort = sorted(distance.items(), key=lambda item: item[1], reverse=True)

In [54]:
# Получившийся топ пользователей 
# По сути он включает пользователей, посещавших статьи по экономике, и политике, 
# так как "нужные" пользователи посещали только одну или 2 статьи (нужную и в категории политика)
top = [x[0] for x in sort]